In [1]:
!pip install ReliefF
!pip install category_encoders

     |████████████████████████████████| 48 kB 1.8 MB/s 
  Created wheel for ReliefF: filename=ReliefF-0.1.2-py3-none-any.whl size=6318 sha256=31a17bb1771823d85b481171347ce28808b9f7c5384d7fda88474211e30c44c0
  Stored in directory: /root/.cache/pip/wheels/18/8a/4a/3aada56cb869df4c1b1ed6af509c941a25985d1d1258a2e78c
Successfully built ReliefF
     |████████████████████████████████| 82 kB 397 kB/s 


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB

from ReliefF import ReliefF

import category_encoders as ce

from scipy.stats import chi2_contingency

np.random.seed(1)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
#https://www.kaggle.com/klmsathishkumar/predict-your-bmi-here
def ordinal_encoding(df,col,mapping):
    ordinal_encoder = ce.OrdinalEncoder(cols = [col],return_df = True,mapping = [{'col':col,'mapping':mapping}])
    df_final = ordinal_encoder.fit_transform(df)
    return df_final

# Import Data

In [5]:
data_stroke = pd.read_csv('Datasets/stroke/Stroke.csv')
data_stroke = data_stroke.dropna()
data_stroke =  data_stroke.drop(columns = ['id'])
data_stroke

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5104,Female,13.0,0,0,No,children,Rural,103.08,18.6,Unknown,0
5106,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [6]:
mapping_gender = {"Male":0,"Female":1, "Other":2}
mapping_marry = {"Yes":0,"No":1}
mapping_work = {"Private":0,"Self-employed":1,"Govt_job":2,"children":3,"Never_worked":4}
mapping_residence = {"Urban":0,"Rural":1}
mapping_smoke = {"formerly smoked":0,"never smoked":1,"smokes":2,"Unknown":3}

data_stroke["gender"] = ordinal_encoding(data_stroke['gender'],"gender",mapping_gender)
data_stroke["ever_married"] = ordinal_encoding(data_stroke['ever_married'],"ever_married",mapping_marry)
data_stroke["work_type"] = ordinal_encoding(data_stroke['work_type'],"work_type",mapping_work)
data_stroke["Residence_type"] = ordinal_encoding(data_stroke['Residence_type'],"Residence_type",mapping_residence)
data_stroke["smoking_status"] = ordinal_encoding(data_stroke['smoking_status'],"smoking_status",mapping_smoke)

In [7]:
data_stroke

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,67.0,0,1,0,0,0,228.69,36.6,0,1
2,0,80.0,0,1,0,0,1,105.92,32.5,1,1
3,1,49.0,0,0,0,0,0,171.23,34.4,2,1
4,1,79.0,1,0,0,1,1,174.12,24.0,1,1
5,0,81.0,0,0,0,0,0,186.21,29.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
5104,1,13.0,0,0,1,3,1,103.08,18.6,3,0
5106,1,81.0,0,0,0,1,0,125.20,40.0,1,0
5107,1,35.0,0,0,0,1,1,82.99,30.6,1,0
5108,0,51.0,0,0,0,0,1,166.29,25.6,0,0


##CHI2

In [8]:
#Aqui solo aparece la ultima comparacion, pero esta se hizo con todas las columnas de la tabla
chi_stroke = data_stroke
ct = pd.crosstab(data_stroke.smoking_status,data_stroke.stroke, margins=True)
ct

stroke,0,1,All
smoking_status,,,
0,780,57,837
1,1768,84,1852
2,698,39,737
3,1454,29,1483
All,4700,209,4909


In [9]:
obs = np.array([ct.iloc[0][0:5].values,
                  ct.iloc[1][0:5].values])
obs

array([[ 780,   57,  837],
       [1768,   84, 1852]])

In [10]:
#Si el 2do valor es menor que 0.05, se puede quedar la columna
chi2_contingency(obs)[0:3]

(6.001667075463959, 0.04974558625866806, 2)

In [11]:
chi_stroke = chi_stroke.drop(columns = ['stroke'])
chi_stroke = chi_stroke.drop(columns = ['gender'])
chi_stroke = chi_stroke.drop(columns = ['work_type'])
chi_stroke = chi_stroke.drop(columns = ['Residence_type'])
#chi_stroke = chi_stroke.drop(columns = ['age'])
#chi_stroke = chi_stroke.drop(columns = ['avg_glucose_level'])
#chi_stroke = chi_stroke.drop(columns = ['bmi'])

chi_stroke

,age,hypertension,heart_disease,ever_married,avg_glucose_level,bmi,smoking_status
0,67.0,0,1,0,228.69,36.6,0
2,80.0,0,1,0,105.92,32.5,1
3,49.0,0,0,0,171.23,34.4,2
4,79.0,1,0,0,174.12,24.0,1
5,81.0,0,0,0,186.21,29.0,0
...,...,...,...,...,...,...,...
5104,13.0,0,0,1,103.08,18.6,3
5106,81.0,0,0,0,125.20,40.0,1
5107,35.0,0,0,0,82.99,30.6,1
5108,51.0,0,0,0,166.29,25.6,0


##RELIEF

In [12]:


y = np.array(data_stroke['stroke'])

data_stroke =  np.array(data_stroke.drop(columns = ['stroke']))


fs = ReliefF(n_neighbors=5, n_features_to_keep=7)

relief_stroke = fs.fit_transform(data_stroke, y)
print(relief_stroke)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 2.]
 ...
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 3.]]


In [13]:
relief_stroke = pd.DataFrame(relief_stroke)
relief_stroke

,0,1,2,3,4,5,6
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,2.0
3,0.0,1.0,0.0,1.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
4904,0.0,0.0,1.0,3.0,1.0,1.0,3.0
4905,0.0,0.0,0.0,1.0,1.0,0.0,1.0
4906,0.0,0.0,0.0,1.0,1.0,1.0,1.0
4907,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Preprocess

In [14]:
scaler = StandardScaler()
relief_stroke = scaler.fit_transform(relief_stroke)
chi_stroke = scaler.fit_transform(chi_stroke)

#Split Data

In [15]:
X_train_relief, X_test_relief, y_train_relief, y_test_relief = train_test_split(relief_stroke, y, test_size=0.33, random_state=0)
X_train_chi, X_test_chi, y_train_chi, y_test_chi = train_test_split(chi_stroke, y, test_size=0.33, random_state=0)

# Models

In [16]:
discriminative = LogisticRegression(random_state = 0)
generative = GaussianNB()
    
discriminative.fit(X_train_relief, y_train_relief)
generative.fit(X_train_relief, y_train_relief)
    
dis_relief = discriminative.score(X_test_relief, y_test_relief)
gen_relief = generative.score(X_test_relief, y_test_relief)

In [17]:
discriminative = LogisticRegression(random_state = 0)
generative = GaussianNB()
    
discriminative.fit(X_train_chi, y_train_chi)
generative.fit(X_train_chi, y_train_chi)
    
dis_chi = discriminative.score(X_test_chi, y_test_chi)
gen_chi = generative.score(X_test_chi, y_test_chi)

In [18]:
print (f'Dis_relief: {dis_relief}\nDis_chi: {dis_chi}\nGen_relief: {gen_relief}\nGen_chi: {gen_chi}\n')

Dis_relief: 0.9617283950617284
Dis_chi: 0.9617283950617284
Gen_relief: 0.8672839506172839
Gen_chi: 0.8598765432098765

